In [2]:
import pandas as pd
import numpy as np
%matplotlib inline

In [3]:
data_path = '../data/'
listing_info_fn = 'listing_info.csv'
test_fn = 'test.csv'
train_fn = 'train.csv'
user_behavior_logs_fn = 'user_behavior_logs.csv'
user_info_fn = 'user_info.csv'
user_repay_logs_fn  = 'user_repay_logs.csv'
user_taglist_fn = 'user_taglist.csv'

preprocess_listing_info_fn = 'preprocess_listing_info.csv'
preprocess_test_fn = 'preprocess_test.csv'
preprocess_train_fn = 'preprocess_train.csv'
preprocess_user_behavior_logs_fn = 'preprocess_user_behavior_logs.csv'
preprocess_user_info_fn = 'preprocess_user_info.csv'
preprocess_user_repay_logs_fn  = 'preprocess_user_repay_logs.csv'
preprocess_user_taglist_fn = 'preprocess_user_taglist.csv'

preprocess_train_data_listing_info_fn = 'preprocess_train_data_listing_info.csv'

In [4]:
user_info = pd.read_csv(data_path+preprocess_user_info_fn)
user_info.drop(['Unnamed: 0'], axis=1,inplace=True)

In [59]:
import copy
user_info = pd.read_csv(data_path+preprocess_user_info_fn)
user_info.drop(['Unnamed: 0'], axis=1,inplace=True)
user_info = pd.concat([user_info, pd.DataFrame(columns=['insertdata_first','insertdata_last', 'insertdata_times'])], sort=True)
user_info_copy = user_info.copy(deep=True)
user_info_copy.head()

,age,cell_province,gender,id_city,id_province,insert_ts,insertdata_first,insertdata_last,insertdata_times,insertdate,reg_mon,user_id
0,19.0,2.0,1.0,295.0,5.0,1.544486e+09,NaN,NaN,NaN,2018-12-11,2017-04,483833.0
1,31.0,20.0,1.0,141.0,20.0,1.518480e+09,NaN,NaN,NaN,2018-02-13,2016-05,156772.0
2,34.0,29.0,1.0,19.0,29.0,1.534810e+09,NaN,NaN,NaN,2018-08-21,2016-05,173388.0
3,25.0,22.0,0.0,115.0,22.0,1.528157e+09,NaN,NaN,NaN,2018-06-05,2016-07,199107.0
4,23.0,26.0,1.0,65.0,26.0,1.522627e+09,NaN,NaN,NaN,2018-04-02,2016-03,122560.0


In [ ]:
handled_index_list = []
join_serise_list = []
count = 0
join_index = ['index', 'insertdata_first', 'insertdata_last', 'insertdata_times']
join_serise = pd.Series(index=join_index)
for idx, row in user_info.iterrows():
    count += 1
    if count%1000 == 0:
        print(count)
#         if count == 1000:
#             break
#     try:
    temp_user = user_info_copy.loc[user_info_copy['user_id'] == row['user_id']]
    temp_user_insterdata_times = len(temp_user)
    if temp_user_insterdata_times == 0:
        print('error')
        break
    if idx in handled_index_list:
        continue
    if temp_user_insterdata_times == 1:   
        temp_series = pd.Series([idx, temp_user['insert_ts'].values[0], temp_user['insert_ts'].values[0], 1], index=join_index)
#             join_serise = pd.concat([join_serise, temp_series], axis=1)
        join_serise_list.append(temp_series)
        continue
    else:
        handled_index_list.append(idx)
    temp_user_ts = temp_user['insert_ts']
    temp_user_ts_max = temp_user_ts.max()
    temp_user_ts_min = temp_user_ts.min()
    temp_user_ts_dict = {v:k for k, v in temp_user_ts.to_dict().items()}
    temp_user_ts_max_idx = temp_user_ts_dict[temp_user_ts_max]
    temp_user_index_list = temp_user.index.to_list()
    temp_user_index_list.remove(temp_user_ts_max_idx)
    user_info_copy.drop(index=temp_user_index_list, axis=0, inplace=True)
    temp_series = pd.Series([temp_user_ts_max_idx, temp_user_ts_min, temp_user_ts_max, temp_user_insterdata_times], index=join_index)
    join_serise_list.append(temp_series)
#         join_serise = pd.concat([join_serise, temp_series], axis=1)
#     except:
#         print(row['user_id'])
#         break
join_serise = pd.concat(join_serise_list, axis=1)
join_serise.head()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [40]:
user_info.shape

(954209, 12)

In [38]:
user_info_copy.shape

(928195, 12)

In [50]:
user_info.loc[user_info['user_id'] == 653719]

,age,cell_province,gender,id_city,id_province,insert_ts,insertdata_first,insertdata_last,insertdata_times,insertdate,reg_mon,user_id
949888,36.0,14.0,1.0,23.0,29.0,1.553472e+09,NaN,NaN,NaN,2019-03-25,2017-09,653719.0
954201,35.0,14.0,1.0,23.0,29.0,1.522109e+09,NaN,NaN,NaN,2018-03-27,2017-09,653719.0


In [ ]:
user_info_copy.drop(index=['insertdata_first', 'insertdata_last', 'insertdata_times'], axis=1, inplace=True)
pd.merge(user_info_copy, join_serise, left_index=True, right_index=True)

In [31]:
join_serise = pd.read_csv(data_path+'join_serise.csv', encoding='utf-8')

In [34]:
join_serise.reset_index(drop=True, inplace=True)
join_serise.drop(index=0, inplace=True)
join_serise['index'] = join_serise['index'].astype(dtype='int64',copy=False)
join_serise.set_index(['index'], inplace=True)

In [17]:
join_serise.to_csv(data_path+'preprocess_join_serise.csv', index=False)

In [48]:
s1 = pd.Series([1,1.524614e+09,1.524614e+09,3], index=['index', 'insertdata_first', 'insertdata_last', 'insertdata_times'])
s2 = pd.Series([2,1.523121e+09,1.624614e+09,2], index=['index', 'insertdata_first', 'insertdata_last', 'insertdata_times'])

In [44]:
new_s = pd.concat([s1, s2], axis=1).T
new_s

,index,insertdata_first,insertdata_last,insertdata_times
0,1.0,1.524614e+09,1.524614e+09,3.0
1,2.0,1.523121e+09,1.624614e+09,2.0


In [45]:
new_s['index'] = new_s['index'].astype(dtype='int64',copy=False)
new_s

,index,insertdata_first,insertdata_last,insertdata_times
0,1,1.524614e+09,1.524614e+09,3.0
1,2,1.523121e+09,1.624614e+09,2.0


In [46]:
new_s.set_index(['index'], inplace=True)
new_s

,insertdata_first,insertdata_last,insertdata_times
index,,,
1,1.524614e+09,1.524614e+09,3.0
2,1.523121e+09,1.624614e+09,2.0
